# Wave notebook tutorial

This tutorial assumes you already know a bit about WAVE and the role of Entities and DoTs.

Firstly, a bit about the example problem we will be solving. In buildings, there is the notion of a *controller*.

![a typical controller](images/controller.png)

This consumes a bunch of information, such as sensor feeds, building models, setpoints and other parameters. Based on that, it then actuates some other devices such as HVAC subsystems. 

In WAVE, these controllers are typically run in *Spawnpoint* which is basically a container management engine that is controllable over WAVE, allowing computing resources to be treated with the same security model as any other resource in WAVE, including delegation and cross-domain federation.

For the purposes of this tutorial, we will be building a simple controller that actuates a light based on some sensors, such as a switch.

### Verify we are connected to the block chain

WAVE makes use of a global blockchain. This command will print out various operating parameters:

In [ ]:
 !bw2 status

A little explanation. Peer count is how many peers we are connected to in order to receive updates to the blockchain. The current block and current age are an indication of how up-to-date our copy of the blockchain is, and the difficulty is a proxy for how many people are mining on the chain.

### Make an entity

As you recall, an entity is the principal in WAVE, a bit like a username and password. We need to create a new entity that you can use to interact with WAVE resources.

We can add additional metadata to this entity, including
- Contact information (`-c`)
- Descriptive message (`-m`)
- Expiration (`-e`, defaults to 30 days)

This keypair is how we identify ourselves to WAVE resources. All WAVE operations (such as subscribing and publishing) are transparently verified by the routing infrastructure because they have a self-contained authenticating proof. This proof is generated by the WAVE "agent" running in this container, so you as a user will never have to interact directly with that construction.

Any operation that interacts with the blockchain can **take a long time to complete** as for security reasons, you need to not only wait for the operation to appear in a block, but also for a few *confirmations* which are blocks after that block that add weight to that branch of the chain. This waiting can be skipped, but WAVE does the secure thing by default and monitors for confirmations.

In [ ]:
!bw2 mke -o myentity.ent -c "Your Name Here" -m "Rise Camp 2017"

### Gain access to a namespace

You have a unique namespace, the name of which is stored in an environment variable as $NAMEPSACE.

We want to interact with resources on `$NAMESPACE`, but we need to have permission to do so. As we just created your entity above, it should not have permission to interact with the namespace yet:

In [ ]:
!echo Namespace is: $NAMESPACE
!bw2 buildchain -t myentity.ent -u $NAMESPACE/*

The `buildchain` command attempts to build a chain of delegations of trust to see if there is a valid path in the DOT graph between the namespace entity and our own. It inspects the URI given by the `-u` argument to find the VK or alias of the entity responsible for administrating that namespace. Then, it traverses the graph of all DOTS granted by that entity until it finds a path of valid, non-expired DOTs that terminates at the VK for `myentity.ent`. This path is called a "DOT chain". As you can see, we do not have permissions yet.

You can look at the permission structure of your namespace in the [Examine Namespace Notebook](ExamineNamespace.ipynb). You should not see your Entity there because there are no permissions linking it to the namespace yet

Now, we need to ask the owner of the namespace for permission to operate on their namespace. We could use our public key (Entity VK) but that is quite unwieldy. Lets first make a human readable alias for our public key.

To get the VK of our entity, look for the "Entity VK" field of the following output

In [ ]:
!bw2 i myentity.ent

In [ ]:
!bw2 mkalias --long "choose.a.unique.alias.here" --b64 "paste.your.vk.here"

Now, using that alias, lets ask for permission to be granted to our entity:

In [ ]:
!./getaccess choose.a.unique.alias.here69

Go over and re-run the [Examine Namespace Notebook](ExamineNamespace.ipynb) again. You should see your entity there. 

We've scripted this interaction for the purposes of the tutorial, but normally, gaining access to a namespace (or subset thereof) would be the outcome of a conversation with the namespace owner or some other individual with the ability to delegate permissions on that namespace.

The output of the above command will output something like 

```
DOT created
Hash:  Us19whkr4WXXrYojB5ytzKuKK_sqfmNh_2UWiTtj3mw=
Wrote dot to file:  .Us19whkr4WXXrYojB5ytzKuKK_sqfmNh_2UWiTtj3mw=.dot
Waiting for 1 object(s) to publish
Current BCIP set to 2 confirmation blocks or 20 block timeout
```

The `hash` represents the DOT object in the blockchain and is inspectable by the `bw2` command, as we observe below

Re-run this next line until "Registry" is marked **valid** which indicates that the grant is globally visible in the Registry smart contract.

In [28]:
!bw2 i <your dot chain hash here>

/bin/sh: 1: Syntax error: end of file unexpected


When the registry field is valid, we know the DOT object can be seen by everyone, and is ready to be used in a proof. You can see the entities involved in the DOT and the resources it grants permission on. 

The URI here is the *canonical form* of the uri where the namespace alias is replaced by its full public key

### Delegate access

We've already created a couple of entities for you --- `light.ent` and `switch.ent` --- that have permissions on $NAMESPACE, as you will have seen in the examine namespace notebook.

In [ ]:
!bw2 i light.ent

You can also see that it has permissions to interact with resources in the namespace under `/s.light/*`:

In [ ]:
!bw2 bc -t light.ent -u $NAMESPACE/s.light/* -v

This is a DoT chain that has been found (it only consists of a single DoT). Similarly, the switch entity has some permissions too:

In [ ]:
!bw2 i switch.ent
!bw2 bc -t switch.ent -u $NAMESPACE/s.switch/* -v

We now introduce the virtual light, which already subscribes to its control URI `/s.light/light0/slot/state`. The text box shows all the messages the light is receiving. The lastest event shows at the top of the textbox.

In [ ]:
from widgets import Light
light = Light()
light.display()

Now, we can create a Python client with our entity and connect to our local agent.

We initialize the client with the entity file we want to use. We specify the entity file (and not our alias or VK) because the file contains the private signing key (SK) that we need for the agent to create proofs and sign our interactions with WAVE resources. You can publish a "true" string to the light's URL to turn it on or off.

In [ ]:
from client import get_client
myentity_bw_client = get_client(entity='myentity.ent')
import os
namespace = os.environ["NAMESPACE"]
# Turn the light on, change "true" to "false" to turn the light off
myentity_bw_client.publish(light.url(), (64,0,0,1), "true")

We also give you a switch, which publishes to its URI when clicked, and a event log that shows all commands sent to that URL. You can click on each button to issue an on or off command. The switch knows nothing about the light. Think of this as being a stand-in for a more complex sensor used as an input for control

In [ ]:
from widgets import Switch
switch = Switch()
switch.display()

### Writing our service

In WAVE, each process, device and individual has its own key. Because permissions are managed at the level of keys, this gives us fine-grained control over what can talk or listen to what.

To begin, we create a new entity for our service binding the switch to the light.

In [ ]:
!bw2 mke -o service.ent

Wait for the service entity to confirm, and then proceed.

We now create DOTs to grant publish permission on the light and consume permission on the switch to the service entity. Note that these are very specific permissions. If our controller were to be compromised, we don't want the attacker to gain more privileges than absolutely necessary.

It takes a little longer for an Entity to be valid to grant permissions to than it does to confirm it, so if the command below does not succeed (it says the destination VK is unknown), try again after a minute.


In [ ]:
!bw2 mkdot -f myentity.ent -t service.ent -u $NAMESPACE/s.light/light0/i.boolean/slot/state -x P

In [ ]:
!bw2 mkdot -f myentity.ent -t service.ent -u $NAMESPACE/s.switch/switch0/i.boolean/signal/state -x C

Now, let us make out mini controller. Typically this kind of logic gets pushed into a persistent process management system like Spawnpoint, but here we are using a notebook as an execution environment.  

Lets start simple. Lets say that our light turns on when the switch is on, and turns off when the switch is off.

In [ ]:
service_bw_client = get_client(entity='service.ent')

print "creating controller consuming: ",switch.url()
print "that actuates: ",light.url()

# This callback gets executed upon a new WAVE message
def service_callback(msg):
    # to help with discovery and interoperability, there is a heirarchical type system
    # in WAVE payloads. This below says the payload is a human-readable UTF8 string
    data_schema=(64,0,0,1)
    if msg.payload == "true":
        #lets turn on the light
        service_bw_client.publish(light.url(), data_schema, "true")
    elif msg.payload == "false":
        service_bw_client.publish(light.url(), data_schema, "false")

# this subscribe call will call the above callback whenever a message is published to the switch URI.
service_bw_client.subscribe(switch.url(), service_callback)

Now you can go back to use the switch to control the light.

If you want to monitor what messages are being sent within your namespace, you can open a terminal using jupyter and run
```
bw2 subscribe --entity myentity.ent $NAMESPACE/*
```

# Part Two: federation

You may be thinking this system is a bit overkill for connecting a light and switch on the same computer. You are right! The real strength of WAVE is that it enforces authorization among multiple parties without relying on a central authority as a mediator. This is completely unlike most authorization systems you are familiar with (e.g OAUTH, LDAP etc).

So now lets try a "Controller As A Service". Controlling your own lights is such a hassle, why not get a third party to do it? Pair up with the person next to you and decide who is the "Controller" and who is the "Client".

### Controllers

You need to have your alias handy to give to your clients, they will need this to grant you permissions

### Clients

To grant permission to let the controller actuate your light, you need to do something like:

In [ ]:
!bw2 mkdot -f myentity.ent -t <controller alias> -u $NAMESPACE/s.light/light0/i.boolean/slot/state -x P --ttl 3

After you do that and it confirms, take a look at your namespace graph, the controller's entity should now be present

### Controllers

Now that you have permission, it's time to follow through on your business model and do some controlling. Enter your client's namespace in below and run the cell to update your controller logic:

In [ ]:
service_bw_client = get_client(entity='service.ent')
target = "enter client's namespace here" + "/s.light/light0/i.boolean/slot/state"
print "creating controller consuming: ",switch.url()
print "that actuates: ",target

def service_callback(msg):
    data_schema=(64,0,0,1)
    if msg.payload == "true":
        service_bw_client.publish(target, data_schema, "true")
    elif msg.payload == "false":
        service_bw_client.publish(target, data_schema, "false")

service_bw_client.subscribe(switch.url(), service_callback)

To gain an edge in the cut-throat switch-as-a-service (SAAS) market, you want to introduce some automation. Whenever it gets cloudy, we want to automatically turn on our lights. So now our controller logic would be:

If it is cloudy: turn on the lights
Otherwise:
   If the switch is on, turn on the lights
   Otherwise turn off the lights
   
So now we need access to a data feed telling us if it is cloudy or not. Fortunately there is another service we can use at `cloudcomputing/s.cloud/_/i.cover/signal/percent`. Lets change our controller to use that feed as well:

In [ ]:
service_bw_client = get_client(entity='service.ent')
target = "enter client's namespace here" + "/s.light/light0/i.boolean/slot/state"
cloudsrc= "cloudcomputing/s.cloud/_/i.cover/signal/percent"
print "creating controller consuming: ",switch.url(), "and",cloudsrc
print "that actuates: ",target

def refresh():
    global itIsCloudy
    global switchIsOn
    data_schema=(64,0,0,1)
    if isIsCloudy or switchIsOn:
        service_bw_client.publish(target, data_schema, "true")
    else:
        service_bw_client.publish(target, data_schema, "false")
        
def service_callback(msg):
    if msg.payload == "true":
        switchIsOn = True
    elif msg.payload == "false":
        switchIsOn = False
    refresh()
    
def cloud_callback(msg):
    global itIsCloudy
    if int(msg.payload) > 70 # super cloudy
        itIsCloudy = true
    else:
        itIsCloudy = false
    refresh()
    
service_bw_client.subscribe(switch.url(), service_callback)
service_bw_client.subscribe(cloudsrc, cloud_callback)

# Part Three: teardown

As a client, you got unhappy with the quality of service you received, and now want to cut ties with your SAAS provider. To audit your namespace, take a look at your [Examine Namespace Notebook](ExamineNamespace.ipynb) and re-run it. Find the Hash for the DOT you granted to the controller (or find it in the output of the mkdot command above). To revoke this permission, run

In [ ]:
!bw2 revoke --from myentity.ent --dot <enter hash here>

Once that completes, rerun the examine namespace notebook again, and you should see that link has disappeared.